In [ ]:
!unzip combined_datasets_withokinawa.zip > /dev/null

In [ ]:
!zip -r labels.zip images_fix/labels/

In [ ]:
!pip install ultralytics > /dev/null

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11n.yaml")  # build a new model from YAML
model = YOLO("yolo11n.pt")  # load a pretrained model (recommended for training)
model = YOLO("yolo11n.yaml").load("yolo11n.pt")  # build from YAML and transfer weights

In [ ]:
!pwd

In [ ]:
# Train the model
model.train(data="/content/combined_datasets/adult_child.yaml", epochs=80, imgsz=640)


In [ ]:
best_model = YOLO("runs/detect/train2/weights/best.pt")

In [ ]:
from IPython.display import Image

Image(filename="runs/detect/train2/results.png", width=800)

In [ ]:
# results = best_model.predict(source="screenshot.png", save=True)

In [ ]:
results = best_model.predict(source="/content/dataset_split/images/test/", save=True)

In [ ]:
!unzip okinawa_202510.zip > /dev/null

In [ ]:
!unzip combined_datasets.zip > /dev/null

In [ ]:
# fine tuning
import os, random, shutil
from pathlib import Path

combined_dir = "finetuning_split"
output_dir = "finetuning_split_"
split_ratio = 0.8  # train:val = 8:2
seed = 42
random.seed(seed)

def main():
    imgs = [f for f in os.listdir(os.path.join(combined_dir, "images")) if f.endswith(".jpg")]
    random.shuffle(imgs)

    n_train = int(len(imgs) * split_ratio)
    train_imgs = imgs[:n_train]
    val_imgs = imgs[n_train:]

    for split, files in [("train", train_imgs), ("val", val_imgs)]:
        os.makedirs(os.path.join(output_dir, "images", split), exist_ok=True)
        os.makedirs(os.path.join(output_dir, "labels", split), exist_ok=True)

        for img_file in files:
            base = Path(img_file).stem
            lbl_file = base + ".txt"

            shutil.copy(os.path.join(combined_dir, "images", img_file),
                        os.path.join(output_dir, "images", split, img_file))
            shutil.copy(os.path.join(combined_dir, "labels", lbl_file),
                        os.path.join(output_dir, "labels", split, lbl_file))

    print(f"train: {len(train_imgs)}, val: {len(val_imgs)}")

if __name__ == "__main__":
    main()


In [ ]:
from ultralytics import YOLO

In [ ]:
best_model = YOLO("20250924_best.pt")

In [ ]:
# Train the model
best_model.train(data="/content/combined_datasets/adult_child.yaml", epochs=20, lr0=0.0005, imgsz=640)


In [ ]:
from IPython.display import Image

Image(filename="runs/detect/train/results.png", width=800)